In [1]:
import pandas as pd
import os

In [2]:
root = os.getcwd()

In [3]:
df = pd.read_csv(root + '/../yelp/processed.csv')

In [4]:
business = pd.read_json(root + '/../yelp/yelp_academic_dataset_business.json',lines=True)

In [5]:
# assume current location
# assume user id 
lat, lng = (29.954163689899758, -90.10036833780715)
id_ = 'fQTgtv3VsocshUiPDBCOEQ'

# get restaurant the user went to before
restaurant_id = df[df.user_id == id_].business_id.to_list()

In [6]:
# get each restaurant location
info = []
for i in restaurant_id:
    info.append(business[business.business_id == i][['name','latitude', 'longitude','categories','attributes']].iloc[0].to_dict())
info

[{'name': 'Please-U-Restaurant',
  'latitude': 29.9367378,
  'longitude': -90.0776308,
  'categories': 'Cajun/Creole, Breakfast & Brunch, Diners, Restaurants',
  'attributes': {'RestaurantsReservations': 'True',
   'RestaurantsGoodForGroups': 'True',
   'RestaurantsPriceRange2': '1',
   'OutdoorSeating': 'False',
   'RestaurantsTakeOut': 'True',
   'Caters': 'True',
   'BikeParking': 'True',
   'GoodForKids': 'True',
   'RestaurantsAttire': "u'casual'",
   'Alcohol': "u'beer_and_wine'",
   'BusinessParking': "{'garage': False, 'street': True, 'validated': False, 'lot': False, 'valet': False}",
   'HasTV': 'True',
   'BusinessAcceptsCreditCards': 'True',
   'WiFi': "u'free'",
   'GoodForMeal': "{'dessert': False, 'latenight': False, 'lunch': True, 'dinner': False, 'brunch': True, 'breakfast': True}",
   'NoiseLevel': "u'average'",
   'RestaurantsDelivery': 'True',
   'Ambience': "{'touristy': False, 'hipster': False, 'romantic': False, 'divey': True, 'intimate': False, 'trendy': False, 

In [7]:
user = df[df.user_id == id_]

In [8]:
import geopandas as gpd
from shapely import Point
from pyproj import CRS
crs = CRS('EPSG:4326')

coord = []
for i in user[['latitude', 'longitude']].values:
    point = Point(i[0],i[1])
    coord.append(point)
user['geometry'] = coord

gdf = gpd.GeoDataFrame(user)

/tmp/ipykernel_655805/4234004129.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user['geometry'] = coord


In [9]:
# Use terrain map layer to see volcano terrain
import folium
m = folium.Map((lat,lng), zoom_start=10)

In [10]:
# Create a geometry list from the GeoDataFrame
geo_df_list = [[point.xy[0][0], point.xy[1][0]] for point in gdf.geometry]

for i, coordinates in enumerate(geo_df_list):
    # Place the markers with the popup labels and data
    lat_, lng_ = coordinates[0], coordinates[1]
    folium.Marker(
        [lng_,lat_],
        popup=
            "review_star: " + str(gdf.review_star.iloc[i]) + "<br>"
            + "name: " + str(gdf.name.iloc[i]) + "<br>"
            + "business_star: " + str(gdf.business_star.iloc[i]) + "<br>"
            + "attributes: " + str(gdf.attributes.iloc[i]) + "<br>"
            + "categories: " + str(gdf.categories.iloc[i]),
        icon=folium.Icon(color="%s" % 'red'),
    )

In [11]:
for i in range(gdf.shape[0]):
    folium.Marker([gdf['geometry'].iloc[i].xy[0][0], gdf['geometry'].iloc[i].xy[1][0]],
        popup=
            "review_star: " + str(gdf.review_star.iloc[i]) + "<br>"
            + "name: " + str(gdf.name.iloc[i]) + "<br>"
            + "business_star: " + str(gdf.business_star.iloc[i]) + "<br>"
            + "attributes: " + str(gdf.attributes.iloc[i]) + "<br>"
            + "categories: " + str(gdf.categories.iloc[i]),
        icon=folium.Icon(color="%s" % 'red'),).add_to(m)
folium.Marker([lat, lng],
             tooltip='current location',
             icon = folium.Icon(color='%s' % 'green')).add_to(m)
m

In [12]:
m = folium.Map((lat,lng), zoom_start=10)

In [13]:
df = pd.read_csv('final.csv').reset_index(drop = True)

In [14]:
def dist(c1,c2):
    import geopy.distance
    return geopy.distance.geodesic(c1, c2).m

In [15]:
distance = []
for i in df[['latitude', 'longitude']].values:
    distance.append(dist((lat,lng), (i[0],i[1])))
df['distance'] = distance
business_id = df.head(3).sort_values(by='distance', ascending=True).business_id.tolist()

In [16]:
a = df.head(3).sort_values(by='distance', ascending=True)

In [17]:
a

,user_id,business_id,latitude,longitude,predicted_img_class,predicted_rating,predicted_sentiment,review_count,distance
1,FxsM4PP3EHty06aS4GK5hg,C26xmwM2_OgJJi2BbL3KkQ,29.954392,-90.068741,meat,4.175303,1,954,3053.100538
0,Zr8dKzULkPshzxrU9MoNwg,3WU1ZobAqXQ07xYoKE2Vyg,29.953143,-90.067791,meat,4.123237,1,1437,3146.788217
2,DAeC-Y66n-jNMJJw4oAfNQ,y-g8sgT7WkN16BBe9Vj41g,29.942194,-90.066132,meat,4.095234,1,529,3561.444579


In [18]:
folium.Marker([a.latitude[0], a.longitude[0]],
             tooltip='recommended restaurant',
             popup= a.business_id[0],
             icon = folium.Icon(color='%s' % 'red')).add_to(m)
folium.Marker([lat, lng],
             tooltip='current location',
             icon = folium.Icon(color='%s' % 'green')).add_to(m)
m